In [1]:
import requests
import bs4
import pandas as pd
import time

In [2]:
# Request stock data of Samsung Electronics for the past 1500 days 
price_url = 'https://fchart.stock.naver.com/sise.nhn?symbol=005930&timeframe=day&count=1500&requestType=0'
price_data = requests.get(price_url)

In [3]:
price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml')

In [8]:
item_list = price_data_bs.find_all('item')
item_list
# "date | opening price | high price | low price | closing price | trading volume"

[<item data="20141007|23240|23720|23240|23240|202983"></item>,
 <item data="20141008|22840|23020|22480|22600|294411"></item>,
 <item data="20141010|22000|22280|21980|22100|436749"></item>,
 <item data="20141013|21640|22540|21560|22160|226895"></item>,
 <item data="20141014|22540|22920|22400|22540|331451"></item>,
 <item data="20141015|22560|22900|22400|22500|282872"></item>,
 <item data="20141016|22320|22420|22080|22300|302966"></item>,
 <item data="20141017|22120|22300|21740|21780|325635"></item>,
 <item data="20141020|22300|22340|22020|22120|189535"></item>,
 <item data="20141021|21880|22000|21620|21660|235321"></item>,
 <item data="20141022|21940|22080|21800|22080|222554"></item>,
 <item data="20141023|22260|22260|21800|21880|170118"></item>,
 <item data="20141024|22060|22240|21840|22240|167569"></item>,
 <item data="20141027|22020|22220|21740|21900|167772"></item>,
 <item data="20141028|21800|22000|21720|21820|124877"></item>,
 <item data="20141029|22000|22600|21800|22600|274522"><

In [10]:
# splitting the first data section by section
temp = item_list[0]['data']
temp.split('|') # "date | opening price | high price | low price | closing price | trading volume

['20141007', '23240', '23720', '23240', '23240', '202983']

In [11]:
# Need only the date and the closing price, which are the first and fifth element, respectively

for item in item_list:
    temp_data = item['data']
    datas = temp_data.split('|')
    print(datas[0], datas[4])

20141007 23240
20141008 22600
20141010 22100
20141013 22160
20141014 22540
20141015 22500
20141016 22300
20141017 21780
20141020 22120
20141021 21660
20141022 22080
20141023 21880
20141024 22240
20141027 21900
20141028 21820
20141029 22600
20141030 23620
20141031 24880
20141103 24700
20141104 24340
20141105 24040
20141106 24080
20141107 24120
20141110 25360
20141111 24620
20141112 24600
20141113 24000
20141114 23880
20141117 24100
20141118 24400
20141119 24360
20141120 24200
20141121 24460
20141124 24400
20141125 23800
20141126 24020
20141127 25280
20141128 25740
20141201 25900
20141202 25900
20141203 25960
20141204 26000
20141205 26040
20141208 26380
20141209 26980
20141210 26340
20141211 26100
20141212 25720
20141215 25460
20141216 25580
20141217 25320
20141218 25320
20141219 26560
20141222 26900
20141223 26780
20141224 26860
20141226 27040
20141229 26580
20141230 26540
20150102 26600
20150105 26660
20150106 25900
20150107 26140
20150108 26280
20150109 26280
20150112 26320
20150113 2

In [12]:
# Now, create a dataframe with these data for the fluent analysis

date_list = [] #date
price_list = [] #closing price
for item in item_list:
    temp_data = item['data']
    datas = temp_data.split('|')
    date_list.append(datas[0])
    price_list.append(datas[4])
    
price_df = pd.DataFrame({'closing price':price_list}, index=date_list)
price_df

,closing price
20141007,23240
20141008,22600
20141010,22100
20141013,22160
20141014,22540
...,...
20201105,60300
20201106,60100
20201109,60200
20201110,60200


In [15]:
# function to make a price dataframe

# code: firm/item code
# timeframe: ie. day/week/month
# count: number of data

def make_price_dataframe(code, timeframe, count):
    url = 'https://fchart.stock.naver.com/sise.nhn?requestType=0'
    price_url = url + '&symbol=' + code + '&timeframe=' + timeframe + '&count=' + count
    price_data = requests.get(price_url)
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml')
    item_list = price_data_bs.find_all('item')
    
    date_list = []
    price_list = []
    for item in item_list:
        temp_data = item['data']
        datas = temp_data.split('|')
        date_list.append(datas[0])
        price_list.append(datas[4])

    price_df = pd.DataFrame({code:price_list}, index=date_list)
    
    return price_df

In [16]:
# testing...
make_price_dataframe('005380', 'day', '1500')

,005380
20141007,182000
20141008,178000
20141010,178000
20141013,178500
20141014,176000
...,...
20201105,175000
20201106,170500
20201109,172500
20201110,173500


In [17]:
firmcode_list = ['005930', '005380', '035420', '003550', '034730']

for num, code in enumerate(firmcode_list):
    price_df = make_price_dataframe(code, 'day', '1500')
    if num == 0 :
        total_price = price_df
    else:
      # merge over concat b/c we are combining dataframes horizontally given that the indices are the same
        total_price = pd.merge(total_price, price_df, how='outer', right_index=True, left_index=True)

total_price

,005930,005380,035420,003550,034730
20141007,23240,182000,154219,73600,244000
20141008,22600,178000,157023,74000,245000
20141010,22100,178000,148210,73600,240000
20141013,22160,178500,151415,72300,238500
20141014,22540,176000,151815,71500,239000
...,...,...,...,...,...
20201105,60300,175000,299000,71100,192500
20201106,60100,170500,301000,73300,190500
20201109,60200,172500,298000,72900,193500
20201110,60200,173500,283000,74000,195000


In [19]:
# same procedure of storing this dataframe into an excel file
from google.colab import files
uploaded = files.upload()

Saving data.xls to data.xls


In [20]:
code_data = pd.read_excel('data.xls')
code_data = code_data[['종목코드', '기업명']] # Item Code, Company Name

def make_code2(x):
    x = str(x)
    return '0' * (6-len(x)) + x # this time, no 'A' in front as observed from URL

code_data['종목코드'] = code_data['종목코드'].apply(make_code2)
code_data

,종목코드,기업명
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템
...,...,...
2381,189980,흥국에프엔비
2382,000540,흥국화재
2383,003280,흥아해운
2384,037440,희림


In [ ]:
for num, code in enumerate(code_data['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            price_df = make_price_dataframe(code, 'day', '1500')
        except requests.exceptions.Timeout:
            time.sleep(60)
            price_df = make_price_dataframe(code, 'day', '1500')
        if num == 0 :
            total_price = price_df
        else:
            total_price = pd.merge(total_price, price_df, how='outer', right_index=True, left_index=True)
    except ValueError:
        continue
    except KeyError:
        continue

total_price.index = pd.to_datetime(total_price.index) # editing the index using .to_datetime function to make index look better
total_price.to_excel(r'C:\Users\User\OneDrive\Desktop\QuantStudy\Quant Strategy with Python\엑셀 데이터\주식 책 데이터\price data.xlsx')